In [1]:
import os 
import requests 
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display 
from openai import OpenAI 

In [2]:
load_dotenv(override=True)
api_key = "OPENAI_API_KEY"
openai = OpenAI()

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website_scraper:
    def __init__(self,url):
        self.url = url
        response = requests.get(url, headers = headers)
        soup = BeautifulSoup(response.content,"html.parser")
        self.title = soup.title.string if soup.text else "No title found"

        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator="\n", strip = True)
        
    

In [4]:
system_prompt = "You are an assistant that analyzes and find information from scraped webpages text\
the text from the webpage is scraped and given to you as prompt, it is important that you don't give any wrong information, if you can't find the information in the text say didn't find\
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [5]:
def user_prompt_for(website_scraper):
    user_prompt = f"this is the website title - \n {website_scraper.title} \n"
    user_prompt += f"this is the scraped website text -\n {website_scraper.text} \n"
    user_prompt += "\n \
please provide important contact of this website in markdown.\
if it includes methos for contacting, then summarize these too.\n\n"
    return user_prompt

In [7]:
# ed = Website_scraper("https://and-element.com/")
# print(ed.title)
# print(ed.text)
# print(user_prompt_for(ed))

In [13]:
def prompt_payload(website_scraper):
    return [
        {"role":"system","content":system_prompt},
        {"role":"user","content":user_prompt_for(website_scraper)}
    ]

In [15]:
def gpt_response(url):
    website_scraper = Website_scraper(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = prompt_payload(website_scraper)
    )
    return response.choices[0].message.content

In [18]:
def display_gpt_response(url):
    response = gpt_response(url)
    display(Markdown(response))

In [19]:
display_gpt_response("https://and-element.com/")

# Contact Information for &Element

## Address
&Element  
S9, Innovation Centre  
Boundary Road  
Colchester  
Essex, CO4 3ZQ

## Phone
01206 259355

## Email
[email protected]

## Summary of Contact Methods
- **Phone Consultation**: Available to call for direct inquiries.
- **Free Review Consultation**: Offering a free review of your digital presence to understand issues and strategize for online success.
- **Email**: Can contact via email for inquiries or consultations.